# Zurich Land Cover Classification

This script presents a visualization of training a U-Net classifier on 7 out of 8 available land cover classes of the Zurich dataset, and detecting the unseen class using the following Baseline Method:
## Dropout as a Bayesian Approximation
https://arxiv.org/abs/1506.02142

Data Visualizations are contained in the notebook `Zurich Land Cover Density Forest.ipynb`

## 1. Import Libraries

In [1]:
%matplotlib inline
# python libraries
from matplotlib.patches import Rectangle
import natsort as ns
from multiprocessing import cpu_count
from sklearn import metrics
import sys

# custom libraries
from helpers.helpers import *
from helpers.data_augment import *
from keras_helpers.unet import *
from keras_helpers.helpers import *
from keras_helpers.callbacks import *

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from sklearn.utils import class_weight
from keras.utils import to_categorical
from keras.models import load_model

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras.models import Sequential
from keras.optimizers import Adam

# Meta-Parameters
#base_dir = '/Users/cyrilwendl/Documents/EPFL'  # for local machine
base_dir = '/raid/home/cwendl'  # for guanabana
sys.path.append(base_dir + '/SIE-Master/Code') # Path to density Tree package Tree package

# custom libraries
from density_tree.density_forest import *
from density_tree.helpers import print_density_tree_latex
from helpers.helpers import imgs_stretch_eq
from baselines.helpers import *
from baselines.plots import *

/opt/conda/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14608742313003633077
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 30015488
locality {
  bus_id: 1
}
incarnation: 6609931628623192634
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1"
]


In [2]:
class_to_remove = 7

## 2. Load Images

In [3]:
path = os.getcwd()

im_dir = r'' + path + '/Zurich_dataset/images_tif/'
gt_dir = r'' + path + '/Zurich_dataset/groundtruth/'

im_names = ns.natsorted(os.listdir(im_dir))
gt_names = ns.natsorted(os.listdir(gt_dir))
print("images: %i " % len(im_names))
print("ground truth images: %i " % len(gt_names))

imgs = np.asarray([im_load(im_dir + im_name) for im_name in im_names])
gt = np.asarray([im_load(gt_dir + gt_name) for gt_name in gt_names])

# histogram stretching
imgs_eq = imgs_stretch_eq(imgs)
imgs = imgs_eq  # continue using stretched image

images: 20 
ground truth images: 20 


### 2.1. GT to labels
Next, we need to convert the ground truth (colors) to labels 

In [5]:
legend = OrderedDict((('Background', [255, 255, 255]),
                      ('Roads', [0, 0, 0]),
                      ('Buildings', [100, 100, 100]),
                      ('Trees', [0, 125, 0]),
                      ('Grass', [0, 255, 0]),
                      ('Bare Soil', [150, 80, 0]),
                      ('Water', [0, 0, 150]),
                      ('Railways', [255, 255, 0]),
                      ('Swimming Pools', [150, 150, 255])))

# get class names by increasing value (as done above)
names, colors = [], []
for name, color in legend.items():
    names.append(name)
    colors.append(color)

gt_maj_label = gt_color_to_label(gt, colors)

In [6]:
# Load Images
def flatten(l):
    return [item for sublist in l for item in sublist]


flat_labels = flatten(flatten(gt_maj_label))
print("Unique Labels in GT: ", np.unique(flat_labels))
print(np.shape(gt_maj_label))
gt = gt_maj_label

Unique Labels in GT:  [0. 1. 2. 3. 4. 5. 6. 7. 8.]
(20,)


### 2.2. Get patches

In [8]:
# Get patches
patch_size = 64
stride_train = 64  # has to be <= patch_size
stride_test = 32  # has to be <= patch_size

# ids for training, validation and test sets (0-19)
ids_train = np.arange(0, 12)
ids_val = np.arange(12, 16)
ids_test = np.arange(16, 20)

# get training, test and validation sets
x_train = get_padded_patches(imgs[ids_train], patch_size=patch_size, stride=stride_train)
x_val = get_padded_patches(imgs[ids_val], patch_size=patch_size, stride=stride_train)
x_test = get_padded_patches(imgs[ids_test], patch_size=patch_size, stride=stride_test)
x_test_nostride = get_padded_patches(imgs[ids_test], patch_size=patch_size, stride=patch_size)

y_train = get_gt_patches(gt[ids_train], patch_size=patch_size, stride=stride_train)
y_val = get_gt_patches(gt[ids_val], patch_size=patch_size, stride=stride_train)
y_test = get_gt_patches(gt[ids_test], patch_size=patch_size, stride=stride_test)
y_test_nostride = get_gt_patches(gt[ids_test], patch_size=patch_size, stride=patch_size)

print(x_test.shape)
print(x_test_nostride.shape)
print(y_test_nostride.shape)

(3902, 64, 64, 4)
(964, 64, 64, 4)
(964, 64, 64)


## 3. Keras CNN

Data Split: 
- Training: 12 images
- Validation: 4 images
- Test: 4 images

Tested Architectures: 

| Model | Patch Size | Data Augmentations | Number of Parameters | Testing Precision (avg) | Testing Recall (avg) | Testing f1 score (avg) | Validation / Test accuracy |
| ------- | ------- | ------- | ------- | ------- | ------- |
| U-Net | 64 | Rot 90°, Flipping  | 7,828,200 | 0.87 | 0.858 | 0.86 | t |
| U-Net | 128 | Rot 90°, Flipping  | 7,828,200 | 0.69 | 0.61 | 0.64 | t |
| U-Net | 128 | Rot 90°, Flipping  | 7,828,200 | 0.90 | 0.89 | 0.89 | v |

In [10]:
# create copies of original data
y_train_label = y_train.copy()
y_val_label = y_val.copy()
y_test_label = y_test.copy()

In [11]:
# get class weights
labels_unique = np.unique(y_train.flatten())
print(labels_unique)
class_weights = class_weight.compute_class_weight('balanced', labels_unique, y_train.flatten())
class_weights[0] = 0  # give less weight to background label class
class_weights[5] = 7  # give less weight to bare soil class
class_weights[8] = 7  # give less weight to swimming pool class

print("Class weights:")
for i, w in enumerate(class_weights):
    print("%15s: %3.3f" % (names[i], w))

[0 1 2 3 4 5 6 7 8]
Class weights:
     Background: 0.000
          Roads: 0.757
      Buildings: 0.480
          Trees: 1.508
          Grass: 2.105
      Bare Soil: 7.000
          Water: 2.006
       Railways: 6.211
 Swimming Pools: 7.000


In [12]:
n_classes = 9

# convert to numpy arrays
x_train = np.asarray(x_train)
x_val = np.asarray(x_val)
x_test = np.asarray(x_test)

# make y data categorical
y_train = to_categorical(y_train_label, n_classes)
y_val = to_categorical(y_val_label, n_classes)

# remove class
classes_to_keep = np.asarray([x for x in range(1, n_classes) if x != class_to_remove])

names_keep = np.asarray(names)[classes_to_keep]
names_keep = names_keep.tolist()
print("classes to keep: " + str(names_keep))

y_train = y_train[..., classes_to_keep]
y_val = y_val[..., classes_to_keep]
n_classes = len(classes_to_keep)
class_weights = class_weights[classes_to_keep]

# print shapes of variables
for var in x_train, y_train, x_val, y_val:
    print(np.shape(var))

classes to keep: ['Roads', 'Buildings', 'Trees', 'Grass', 'Bare Soil', 'Water', 'Swimming Pools']
(3897, 64, 64, 4)
(3897, 64, 64, 7)
(1117, 64, 64, 4)
(1117, 64, 64, 7)


### 3.1. Train CNN

In [14]:
# callbacks (evaluated every epoch)
# show loss and accuracy figures after each epoch
callback_plot = PlotLosses()

# stop early if after several epochs the accuracy doesn't improve
callback_earlystop = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=24, verbose=1, mode='auto')

# decrease learning rate when accuracy stops improving
callback_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=12, verbose=1, mode='auto',
                                epsilon=1e-4, cooldown=0, min_lr=1e-8)

# checkpoint to save weights at every epoch (in case of interruption)
file_path = "weights-improvement.hdf5"
callback_checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

callback_tensorboard = TensorBoard(log_dir='./tensorboard', histogram_freq=0, write_graph=True, write_images=True)

# model setup
batch_size = 20
epochs = 300


def model_train(model, data_augmentation):
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(batch_generator(x_train, y_train,
                                        batch_size=batch_size, data_augmentation=data_augmentation),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / float(batch_size))),
                        epochs=epochs,
                        verbose=1,
                        class_weight=class_weights,  # weights for loss function
                        validation_data=(x_val, y_val),
                        callbacks=[callback_earlystop,
                                   callback_lr,
                                   #callback_checkpoint,
                                   callback_plot,
                                   callback_tensorboard],
                        workers=cpu_count(),
                        use_multiprocessing=True)

<Figure size 432x288 with 0 Axes>

In [15]:
# train or load model
# train the model
#model_unet = get_unet(n_classes, x_train.shape[1:])
#model_train(model_unet, data_augmentation=True)
#model_unet.save('models_out/model_unet_64_flip_rot90_wo_cl_' + str(names[class_to_remove]).lower() + '_2.h5')  # save model, weights

In [16]:
# load model
name_model = path + '/models_out/model_unet_64_flip_rot90_wo_cl_' + str(names[class_to_remove]).lower() + '.h5'    
model_unet = load_model(name_model, custom_objects={'fn': ignore_background_class_accuracy(0)})

ResourceExhaustedError: OOM when allocating tensor with shape[512]
	 [[Node: conv2d_10/bias/Assign = Assign[T=DT_FLOAT, _class=["loc:@conv2d_10/bias"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_10/bias, conv2d_9/Const)]]

Caused by op 'conv2d_10/bias/Assign', defined at:
  File "/opt/conda/envs/python36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/envs/python36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/envs/python36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/envs/python36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/opt/conda/envs/python36/lib/

### 3.2. Prediction on Test Set

In [ ]:
# get prediction
y_pred = model_unet.predict(x_test, batch_size=20, verbose=1)

# prediction patches without overlapping patches
y_pred = np.concatenate(remove_overlap(imgs, y_pred, ids_test, 64, 32))

# get label
y_pred_label = get_y_pred_labels(y_pred, class_to_remove=class_to_remove)

# Get accuracy as margin between highest and second highest class
y_pred_acc = get_accuracy_probas(y_pred)

In [ ]:
# prediction image
y_pred_acc_imgs = [convert_patches_to_image(imgs, y_pred_acc[...,np.newaxis],
                                       img_idx=idx_im, img_start=ids_test[0], patch_size=64,
                                       stride=64) for idx_im in ids_test]

### 3.3. Accuracy Metrics (Test Set)

In [ ]:
# Accuracy metrics
y_pred_flattened= np.asarray(flatten(flatten(y_pred_label))).astype('int')
y_test_flattened= np.asarray(flatten(flatten(y_test_nostride))).astype('int')

# mask background and removed classes for evaluation metrics
filter_items = (y_test_flattened != 0) & (y_test_flattened != class_to_remove)

# Class accuracy, average accuracy
print(metrics.classification_report(
    y_test_flattened[filter_items],
    y_pred_flattened[filter_items],
    target_names=names_keep,
    digits=3))


# Overall accuracy
OA = metrics.accuracy_score(y_test_flattened[filter_items], y_pred_flattened[filter_items])
print("Overall accuracy: %.3f %%" % (OA*100))

In [ ]:
# print to log file
import pandas as pd
precision, recall, fscore, support = metrics.precision_recall_fscore_support(y_test_flattened[filter_items], y_pred_flattened[filter_items])
df = pd.DataFrame(data={'Precision':precision,
                        'Recall':recall,
                       'f1-score':fscore,
                       'support':support}, index=names_keep)

df.index.name = 'Class'
with open("models_out/acc_class_" + str(class_to_remove) + ".csv", 'w') as f:
    print(df.to_latex(float_format='%.3f'), file=f)  # Python 3.x

#### CNN accuracies
```python
AA_CNN=[87.098, 81.905, 87.839, 81.076,
        81.236, 83.891, 81.091, 81.451] 
```

## Dropout as a Bayesian Approximation
https://arxiv.org/abs/1506.02142

In [ ]:
# get predictions
y_preds = predict_with_dropout_imgs(model_unet, x_test,
                                    imgs, ids_test, batch_size=100,
                                    n_iter=5)

In [ ]:
prediction = np.mean(y_preds, axis=0)
uncertainty = np.var(y_preds, axis=0)

In [ ]:
np.shape(uncertainty), np.shape(prediction)

In [ ]:
# uncertainty = variance where class prediction is maximum
ind = np.argmax(prediction, axis=-1) # indicator where maximum element lies
uncertainty_new = np.max(prediction, axis=-1) # initialize with same dimensions as label prediction
for patch in range(np.shape(prediction)[0]):
    for w in range(np.shape(prediction)[1]):
        for h in range(np.shape(prediction)[2]):
            uncertainty_new[patch, w, h] = uncertainty[patch, w, h, ind[patch, w, h]]
        
uncertainty = uncertainty_new
# standardize between (0,1)
uncertainty -= np.min(uncertainty)
uncertainty /= np.max(uncertainty)

In [ ]:
np.shape(prediction), np.shape(uncertainty)

In [ ]:
pred_labels = get_y_pred_labels(prediction, class_to_remove=class_to_remove)
pred_labels_im = [convert_patches_to_image(imgs, pred_labels[...,np.newaxis],
                                          i, patch_size=64, stride=64, img_start=16) for i in ids_test]
pred_labels_im = [gt_label_to_color(pred_labels_im[i][...,0], colors) for i in range(len(ids_test))]

plt.imshow(pred_labels_im[0])

In [ ]:
pred_cert_im = [convert_patches_to_image(imgs, uncertainty[..., np.newaxis]
                                         , i, patch_size=64, stride=64, img_start=16) for i in ids_test]

img = imgs_stretch_eq(pred_cert_im[0])
plt.imshow(img[...,0])

In [ ]:
# change shape of y
print(np.shape(y_preds))
y_preds = np.transpose(y_preds, (1,2,3,4,0))
print(np.shape(y_preds))

In [ ]:
# Y (label)
# get prediction
y_pred = model_unet.predict(x_test, batch_size=20, verbose=1)
y_pred = np.concatenate(remove_overlap(imgs, y_pred, ids_test, 64, 32))

# prediction patches without overlapping patches
# y_pred = np.concatenate(remove_overlap(imgs, y_pred, ids_test, 64, 32))

# get label
y_pred_label = get_y_pred_labels(y_pred, class_to_remove=class_to_remove)

y_mlp = np.equal(y_pred_label, y_test_nostride)

In [ ]:
# X (training data)
idx_tr_mlp = np.arange(get_offset(imgs,64,64,16,18))
idx_te_mlp = np.arange(get_offset(imgs,64,64,16,18),get_offset(imgs,64,64,16,20))

x_tr_mlp = y_preds[idx_tr_mlp]
x_te_mlp = y_preds[idx_te_mlp]

y_tr_mlp = y_mlp[idx_tr_mlp]
y_te_mlp = y_mlp[idx_te_mlp]


def cat(var):
    return np.concatenate(np.concatenate(var))

x_tr_mlp = np.transpose(cat(x_tr_mlp),(2,0,1))
x_te_mlp = np.transpose(cat(x_te_mlp),(2,0,1))
y_tr_mlp = cat(y_tr_mlp)
y_te_mlp = cat(y_te_mlp)

In [ ]:
# MLP, similar to baseline 1
# loop all data points

# transformation seem to deteriorate performance
_, x_train_mlp = reorder_truncate_concatenate(x_tr_mlp, n_components=20)
_, x_test_mlp = reorder_truncate_concatenate(x_te_mlp, n_components=20)
#x_train_mlp  = np.concatenate(x_tr_mlp, axis=-1)
#x_test_mlp  = np.concatenate(x_te_mlp, axis=-1)

In [ ]:
# convert class vectors to binary class matrices
num_classes = 2

y_train_mlp = keras.utils.to_categorical(y_tr_mlp, num_classes)
y_test_mlp = keras.utils.to_categorical(y_te_mlp, num_classes)

print(np.shape(y_train_mlp),np.shape(y_test_mlp))
np.shape(x_train_mlp),np.shape(x_test_mlp)

In [ ]:
# Train MLP
model_mlp = Sequential()
model_mlp.add(Dense(300, activation='relu', input_shape=(x_train_mlp.shape[1:])))
model_mlp.add(Dropout(0.5))
model_mlp.add(Dense(300, activation='relu'))
model_mlp.add(Dropout(0.5))
model_mlp.add(Dense(num_classes, activation='softmax'))

model_mlp.summary()

model_mlp.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

batch_size = 2000
epochs = 20

train = True
if train:
    history = model_mlp.fit(x_train_mlp, y_train_mlp,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test_mlp, y_test_mlp))
    model_mlp.save('models_out/model_MLP_BL2_wo_cl_' + str(names[class_to_remove]).lower() + '_2.h5')  # save model, weightsw

In [ ]:
model_mlp = load_model('models_out/model_MLP_BL2_wo_cl_' + str(names[class_to_remove]).lower() + '_2.h5')

In [ ]:
y_pred_proba = model_mlp.predict(x_test_mlp, verbose = 1,
                                 batch_size=batch_size)
y_pred_proba_label = y_pred_proba[:,1]  # probability to have a good label

In [ ]:
# get patches from pixels
n_patches = get_offset(imgs, 64, 64, 18, 20)
y_pred_proba_label_patches = np.reshape(y_pred_proba_label,(n_patches,64, 64))

In [ ]:
# get image from patches
img_idx = 18
y_pred_proba_label_imgs = convert_patches_to_image(imgs, y_pred_proba_label_patches[..., np.newaxis], img_idx, 64, 64, 18)
plt.figure(figsize=(10,10))
plt.imshow(y_pred_proba_label_imgs[...,0])
plt.savefig("../Figures/baseline/BL2_im_" + str(img_idx+1) + "_pred_wo_cl_" + str(class_to_remove) + ".pdf", bbox_inches='tight', pad_inches=0)

In [ ]:
# calculate average certainty by  MLP
av_cert = np.asarray([np.nanmean(y_pred_proba_label_patches
                      [y_test_nostride[idx_te_mlp]==label]) 
           for label in np.arange(1, 9)])
av_cert[-1] = 0


plot_probas(av_cert, class_to_remove, names[1:])
plt.savefig("../Figures/baseline/BL2_probas_" + str(class_to_remove) + ".pdf", bbox_inches='tight', pad_inches=0)
# ratio unseen class / seen classes
cert_unseen = av_cert[class_to_remove - 1]
cert_seen = np.nanmean(np.asarray(av_cert)[av_cert != cert_unseen])

# weighted accuracies
# get support (for weighting)
_, support = np.unique(y_test_nostride, return_counts=True)
support = support[1:]
av_cert_w = (av_cert*support)/sum(support)
cert_unseen_w = av_cert_w[class_to_remove - 1]
cert_seen_w = np.nanmean(np.asarray(av_cert)[av_cert_w != cert_unseen])

print("Average certainty unseen class:\t%.5f" % cert_unseen)
print("Average certainty seen classes:\t%.5f" % cert_seen)
print("Ratio between support-weighted cert. of seen classes / unseen class:\t%.3f" % (cert_seen_w / cert_unseen_w))

In [ ]:
# without transformations
#ratios = [12.750, 11.152, 19.619]
# with transformations (reorder, truncate (n=4))
ratios = [6.266, _, _, _]

# with transformations (reorder, truncate (n=20))
ratios = [7.194, 7.815, 11.071, 4.007, 44.917, 169.341, 144.520, 0]
np.dot(ratios, support)/sum(support)